In [1]:
import os

In [2]:
%pwd

'c:\\ds_projects\\End-to-End-Lung-Cancer-Classification-MLflow-DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\ds_projects\\End-to-End-Lung-Cancer-Classification-MLflow-DVC'

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [20]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    validation_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [21]:
from lung_cancer.constants import *
from lung_cancer.utils.common import read_yaml, create_directories, save_json

In [22]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            validation_data="artifacts/data_ingestion/Data/valid",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config
    
    def get_testing_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            validation_data="artifacts/data_ingestion/Data/test",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [25]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.validation_data,
            #subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        
    def _test_generator(self):
        
        datagenerator_kwargs = dict(
            dtype='float32',
            rescale = 1./255,
        )
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        test_datagenerator  =  tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
        self.test_generator = test_datagenerator.flow_from_directory(
            directory=self.config.validation_data,
            shuffle=False,
            class_mode="categorical",
            **dataflow_kwargs
            )
    

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._test_generator()
        self.score = self.model.evaluate(self.test_generator)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    


In [26]:
try:
    config = ConfigurationManager()
    val_config = config.get_testing_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e

[2023-10-10 17:41:39,263: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-10 17:41:39,267: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-10 17:41:39,267: INFO: common: created directory at: artifacts]
Found 315 images belonging to 4 classes.
20/20 [==============================] - 25s 1s/step - loss: 0.7472 - accuracy: 0.7111
[2023-10-10 17:42:04,367: INFO: common: json file saved at: scores.json]
